<a href="https://colab.research.google.com/github/bioquaintum/pyneurorg/blob/main/examples/00_Installation_and_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation and Setup for pyneurorg

**Objective:** This notebook guides you through the installation of the `pyneurorg` library and its dependencies. It also includes a simple test at the end to verify that `pyneurorg` and its core dependency, Brian2, are functioning correctly.

**Prerequisites:**
- Python 3.7+ (Brian2 typically supports recent Python 3 versions)
- `pip` (Python package installer)

## 1. Understanding `pyneurorg`

`pyneurorg` is a Python module designed for simulating brain organoids, leveraging the power and flexibility of the Brian2 spiking neural network simulator. It provides a structured framework to model neurons and synapses within organoids, simulate their development and activity, and interact with them using simulated Microelectrode Arrays (MEAs) and calcium imaging techniques.

## 2. Installation Options

There are a couple of ways to install `pyneurorg`:

### Option 2.1: Installing from a local clone (Recommended for development or if you have the source code)

If you have cloned the `pyneurorg` repository from GitHub or have the source code folder:

1.  **Navigate to the `pyneurorg` root directory in your terminal:**
    This is the directory that contains the `setup.py` file.
    ```bash
    cd path/to/your/pyneurorg
    ```

2.  **(Recommended) Create and activate a virtual environment:**
    This keeps your project dependencies isolated.
    ```bash
    python -m venv venv_pyneurorg
    # On macOS/Linux:
    source venv_pyneurorg/bin/activate
    # On Windows:
    .\venv_pyneurorg\Scripts\activate
    ```

3.  **Install `pyneurorg` and its dependencies:**
    The `-e` flag installs the package in "editable" mode, meaning changes you make to the source code will be immediately effective without needing to reinstall.
    ```bash
    pip install -e .
    ```
    This command will read the `setup.py` and `requirements.txt` files to install `pyneurorg` and all necessary libraries like Brian2, NumPy, Matplotlib, etc.

### Option 2.2: Installing from PyPI (If `pyneurorg` is published)

If `pyneurorg` is (or will be) published on the Python Package Index (PyPI), you can install it directly using `pip`:

```bash
pip install pyneurorg
```
*(Note: This option is only valid once the package is actually available on PyPI.)*

In [3]:
!pip3 install pyneurorg

## 3. Core Dependencies

`pyneurorg` relies on several key libraries:

- **Brian2:** The core spiking neural network simulator.
- **NumPy:** For numerical operations and array manipulation.
- **Matplotlib:** For plotting and visualization.
- **SciPy:** (Often a dependency of Brian2 or used for advanced analysis) For scientific and technical computing.

These should be automatically installed when you install `pyneurorg` using `pip` (as they should be listed in `requirements.txt` and/or `install_requires` in `setup.py`).

## 4. Verifying Installation (Python Code)

After installation, you can verify that `pyneurorg` and its main dependency, Brian2, can be imported and are functioning. The following cells will attempt to do this.

### 4.1. Import Key Libraries

Let's try importing `brian2` and the main `pyneurorg` package. If these imports work without error, it's a good first sign.

In [4]:
print("Attempting to import Brian2...")
try:
    import brian2 as b2
    print(f"Brian2 imported successfully! Version: {b2.__version__}")
except ImportError as e:
    print(f"Error importing Brian2: {e}")
    print("Please ensure Brian2 is installed correctly. It should be a dependency of pyneurorg.")

print("\nAttempting to import pyneurorg...")
try:
    from pyneurorg.core import neuron_models
    # You might also want to try importing a submodule to be more thorough
    #from pyneurorg.core import neuron_models
    print(f"pyneurorg.core.neuron_models imported successfully!")
    # If pyneurorg has a __version__ attribute (good practice to add one in pyneurorg/__init__.py)
    # if hasattr(pyneurorg, '__version__'):
    #     print(f"pyneurorg version: {pyneurorg.__version__}")
except ImportError as e:
    print(f"Error importing pyneurorg: {e}")
    print("Please ensure pyneurorg is installed correctly (e.g., `pip install -e .` from the root directory) and you are in the correct Python environment.")

Attempting to import Brian2...
Brian2 imported successfully! Version: 2.9.0

Attempting to import pyneurorg...
pyneurorg.core.neuron_models imported successfully!


### 4.2. Quick Test: Creating a Simple Brian2 NeuronGroup via `pyneurorg`

This test will use a predefined neuron model from `pyneurorg` to create a simple `NeuronGroup` using Brian2. This confirms that `pyneurorg`'s model definitions are compatible with Brian2 and that basic Brian2 operations are working.

**Expected Outcome:**
The code should run without errors and print information about the created `NeuronGroup` and a short simulation run.

In [5]:
import numpy as np

print("\nRunning a quick pyneurorg + Brian2 test...")
try:
    # Ensure Brian2 and pyneurorg's neuron_models were imported successfully in the previous cell
    if 'b2' not in locals() or 'neuron_models' not in locals():
        raise RuntimeError("Brian2 or pyneurorg.core.neuron_models not imported. Cannot run test.")

    # 1. Get a neuron model definition from pyneurorg
    print("\n1. Fetching LIFNeuron model definition from pyneurorg.core.neuron_models...")
    lif_model_def = neuron_models.LIFNeuron(
        tau_m=10*b2.ms,
        v_rest=-60*b2.mV,
        v_reset=-70*b2.mV,
        v_thresh=-50*b2.mV
    )
    print("   LIFNeuron model definition fetched.")

    # 2. Set initial conditions and an input current to make them spike
    print("\n2. Setting initial conditions and input current...")
    # The LIFNeuron model expects 'I_input' to be defined (e.g., by synapses)
    # or we can directly set it if it were a parameter.
    # For this test, let's use the 'I_tonic_val' from the namespace,
    # or add a constant current directly if LIFNeuron model_params allowed I_tonic.
    # Modifying lif_model_def to include a default I_tonic for testing:
    lif_model_def_test = neuron_models.LIFNeuron(
        tau_m=10*b2.ms, v_rest=-60*b2.mV, v_reset=-70*b2.mV,
        v_thresh=-50*b2.mV, I_tonic=0.15*b2.nA # Add tonic current
    )
    # Create group with tonic current
    N_test = 5
    test_neurons_active = b2.NeuronGroup(
        N_test,
        model=lif_model_def_test['model'],
        threshold=lif_model_def_test['threshold'],
        reset=lif_model_def_test['reset'],
        refractory=lif_model_def_test['refractory'],
        method=lif_model_def_test['method'],
        namespace=lif_model_def_test['namespace']
    )
    test_neurons_active.v = -70*b2.mV
    print("   Initial conditions set. Tonic current should drive activity.")

    # 3. Setup a SpikeMonitor
    print("\n3. Setting up a SpikeMonitor...")
    spike_mon_test = b2.SpikeMonitor(test_neurons_active)
    print("   SpikeMonitor created.")

    # 4. Create a Network and run for a short duration
    print("\n4. Creating a Brian2 Network and running simulation...")
    test_net = b2.Network(test_neurons_active, spike_mon_test)
    # Set Brian2 preferences for this specific run if not globally set
    b2.prefs.codegen.target = 'numpy' # For simple, non-compiled execution
    test_net.run(50*b2.ms, report='text')
    print("   Simulation run completed.")

    # 5. Check for spikes
    print("\n5. Checking results...")
    if len(spike_mon_test.i) > 0:
        print(f"   SUCCESS! {len(spike_mon_test.i)} spikes were recorded from {len(np.unique(spike_mon_test.i))} neurons.")
        print(f"   Spike times: {spike_mon_test.t}")
    else:
        print("   NOTE: No spikes were recorded. This might be okay if I_tonic was too low or parameters need adjustment for spiking.")
        print("   However, the fact that the simulation ran without Brian2 errors is a good sign.")

    print("\nQuick test finished successfully (simulation ran).")

except Exception as e:
    print(f"\nAN ERROR OCCURRED DURING THE QUICK TEST: {e}")
    import traceback
    traceback.print_exc()


Running a quick pyneurorg + Brian2 test...

1. Fetching LIFNeuron model definition from pyneurorg.core.neuron_models...
   LIFNeuron model definition fetched.

2. Setting initial conditions and input current...
   Initial conditions set. Tonic current should drive activity.

3. Setting up a SpikeMonitor...
   SpikeMonitor created.

4. Creating a Brian2 Network and running simulation...
Starting simulation at t=0. s for a duration of 50. ms
50. ms (100%) simulated in < 1s
   Simulation run completed.

5. Checking results...
   SUCCESS! 10 spikes were recorded from 5 neurons.
   Spike times: <spikemonitor.t: array([16., 16., 16., 16., 16., 34., 34., 34., 34., 34.]) * msecond>

Quick test finished successfully (simulation ran).


## 5. Next Steps

If all the above steps, especially the quick test, completed without critical errors, your `pyneurorg` installation should be ready!

You can now proceed to the next notebooks in the `examples/` directory to learn how to:
- Create more complex organoids (`01_Creating_Your_First_Organoid.ipynb`).
- Run simulations and record different types of data (`02_Running_a_Simple_Simulation_and_Recording_Spikes.ipynb`).
- And much more!